## RQ2 Data Collection


[Add description]

Wikiloc data extraction for Germany

Python code provided by Chai-Allah et al, 2023 through their GitHub repo: https://github.com/achaiallah-hub/Wiki4CES

To run scrapy you need to set up a scrapy "project". In Anaconda Prompt:

1. cd C:\Users\ninam\Documents\UZH\04_Thesis\code\nm_forest_thesis
2. conda activate C:\Users\ninam\Documents\UZH\04_Thesis\code\nm_forest_thesis\thesis_env_conda 
3. scrapy startproject wikiloc_scrapy

Helpful scrapy information: https://docs.scrapy.org/en/latest/intro/tutorial.html

Then you need to add the scrapy "spiders" (the processes which crawl and scrape information from the web) as .py scripts to the directory wikiloc_scrapy/wikiloc_scrapy/spiders. Basically, the .py scripts from the achaiallah-hub repo need to be saved in this directory. In Anaconda Prompt:

1. cd wikiloc_scrapy\wikiloc_scrapy\spiders
2. git init
3. git remote add origin git@github.com:achaiallah-hub/Wiki4CES.git
4. git pull origin main

**NOTE** Instead of Steps 2-4 I first just tried the simple command git clone https://github.com/achaiallah-hub/Wiki4CES.git This works, but it stores the .py scripts inside a repo directory folder called Wiki4CES, and I think this causes problems when trying to use the spiders later. Steps 2-4 are a work-around: this way the python scripts are directly within the "spiders" directory without being inside another directory. In order to do steps 2-4 an **ssh-key** needs to be set up. 

From what I understand, the spiders provided by achaiallah-hub do the following:
1. **extract_link.py** Extracts the URLS for all the trails. You give it a starting point (an intital URL) and it goes through each city/town and extracts all the trail links (URLS) in the cities listing. It stores each link as {"Link": link}. Needs to be run first to get the URLS for steps 2 and 3. 
2. **wikiloc_track.py** Scrapes the trail details like track name, difficulty, distance, author, views and description. It loads the trail URLs from a file called link.csv (presumably created in step 1)
3. **wikiloc_image.py** Scrapes image data from the trail pages, including URL, track name, user name, date, and location (latitude & longitude). It reads the trail pages from a file called link.csv (presumably created in step 1)
4. **download_image.py** Downloads images from the URLS in a csv file called wikiloc_image.csv (presumably this would be created from step 3)

#### Step 1: extract_link.py

Edit the extract_link.py to replace the staring_urls. Originally this contained https://www.wikiloc.com/trails/france/auvergne-rhone-alpes - this URL doesn't seem to exist anymore as it just redirects to https://www.wikiloc.com/trails/outdoor. My guess is that the URL format now needs to be https://www.wikiloc.com/trails/outdoor/ + *country_name* so for Germany I will try https://www.wikiloc.com/trails/outdoor/germany

Then in Anaconda Prompt (with conda environment activated and from the scrapy project's top level directory):
1. scrapy crawl wiki -o link.csv

